# Mass-Spring-Damper System: Postprocessing the DeepONet

In [2]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')

### Importing External Libraries

import numpy                             as np
import pandas                            as pd


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


from scipy.integrate import solve_ivp
import pyDOE
from PCAfold         import PCA          as PCAA


import romnet                            as rmnt

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


## ODE

In [ ]:
def fOrig(t, y):
    return [y[1], -Params[2]/Params[0]*y[0]-Params[1]/Params[0]*y[1]]

def fOrig_Source(t, y):
    return [y[1], -Params[2]/Params[0]*y[0]-Params[1]/Params[0]*y[1]]

Params = np.array([1.0, 0.5, 3.0], dtype=np.float64)

## Defining Input Data

In [ ]:
path_to_run_fld = os.path.join(WORKSPACE_PATH, 'ROMNet/MSD_100Cases_/DeepONet/MSD_TestCase1/')
#path_to_run_fld = os.path.join(WORKSPACE_PATH, 'ROMNet/MSD_100Cases/DeepONet/Run_1/')
FigDir          = os.path.join(WORKSPACE_PATH, '../Desktop/Paper_Figures_DeepONet_TEMP/')


TrainingCases   = [0]#[0,2,4,6,8]
TestCases       = [0,2,4]#[0,2]

NSamples        = 100

Nt              = 100
tout            = np.linspace(0.,15.,Nt)
SOLVER          = 'Radau'


LineVec         = ['-',':','--','.-']*10
ColorVec        = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

## Loading ROMNet's Input Data File

In [ ]:
print("\n[ROMNet]: Reading Input File from: ", path_to_run_fld)
sys.path.insert(0, path_to_run_fld)

print("\n[ROMNet]: Keep Loading Modules and Functions...")
from ROMNet_Input import inputdata

print("\n[ROMNet]: Initializing Input ...")
InputData               = inputdata(WORKSPACE_PATH)


InputData.InputFilePath = path_to_run_fld+'/ROMNet_Input.py'
InputData.train_int_flg = 0
InputData.path_to_run_fld  = path_to_run_fld

## Loading Tensorflow's Model for the Network 

In [ ]:
surrogate_type = InputData.surrogate_type
if (surrogate_type == 'FNN-SourceTerms'):
    surrogate_type = 'FNN'

Net   = getattr(rmnt.architecture, surrogate_type)

model = rmnt.model.Model_TF(InputData)

if (InputData.phys_system is not None):
    System = getattr(rmnt.pinn.system, InputData.phys_system)
    system = System(InputData)
    
model.build(InputData, None, Net, system)#, loadfile_no='000027')

NN    = model.net

## Evaluating DeepONet on Test Data

In [ ]:
input_vars  = model.net.input_vars
trunk_vars  = InputData.input_vars['DeepONet']['Trunk']
branch_vars = InputData.input_vars['DeepONet']['Branch']

FileName  = WORKSPACE_PATH + '/ROMNet/Data/MSD_100Cases/Orig/test/ext/Input.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
Input     = Data[branch_vars].to_numpy()
VarNames  = list(Data.columns)
Nu        = Input.shape[1]
u         = []
for iu in range(Nu):
    u.append(np.unique(Input[:,iu]))
u         = np.stack(u, axis=0).T
u

In [ ]:
### Evaluating on Test Data

FileName  = WORKSPACE_PATH + '/ROMNet/Data/MSD_100Cases/Orig/test/ext/Input.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
Input     = Data[branch_vars].to_numpy()
VarNames  = list(Data.columns)
Nu        = Input.shape[1]
u         = []
for iu in range(Nu):
    u.append(np.unique(Input[:,iu]))
u         = np.stack(u, axis=0).T


## Variable to Be Visualized
VarOI     = 'x'
for iVar in range(1,len(VarNames)):
    if (VarNames[iVar] == VarOI):
        print('iVar = ', iVar)
        break

iSim     = 0
fig, ax1 = plt.subplots()
ax2      = ax1.twinx()  # instantiate a second axes that shares the same x-axis
#for iTest in [0,1,2,3,4]:
for iTest in range(10):
    
    print('u0 = ', u[iTest,:])

    ### Generating Data  
    output     = solve_ivp( fOrig, tout[[0,-1]], u[iTest,:], method=SOLVER, t_eval=tout, rtol=1.e-15, atol=1.e-10 )
    tOrig      = tout
    yMatOrig   = output.y.T
    
    ### Creating NN Input
    tPred     = tOrig[...,np.newaxis]
    Nt        = len(tPred)
    uPred     = u[iTest,:]*np.ones((Nt,2))
    InputPred = np.concatenate((uPred,tPred), axis=1)
    

    yMat      = model.predict(InputPred) 

    tLoop         = np.linspace(0.,15.,3)
    yMatLoop      = np.zeros((len(tLoop),2))
    InputPredLoop = np.concatenate([uPred[0,:][np.newaxis,...], (tLoop[0])[np.newaxis,np.newaxis]], axis=1)
    yMatLoop[0,:] = model.predict(InputPredLoop)
    for it, tLoop_ in enumerate(tLoop[1:]):
        InputPredLoop    = np.concatenate([yMatLoop[it,:][np.newaxis,...], (tLoop[it+1]-tLoop[it])[np.newaxis,np.newaxis]], axis=1)
        yMatLoop[it+1,:] = model.predict(InputPredLoop)

    ## Plotting

    if (iSim == 0):
        Label1 = 'ODE'
        #Label2 = 'DeepONet'
        Label2 = 'PI-DeepONet'
        #Label2 = 'SVD-DeepONet'
        Label3 = 'DeepONet, Iterative'
    else:
        Label1 = None
        Label2 = None
        Label3 = None


    color = ColorVec[1]
    ax1.plot(tOrig, yMatOrig[:,iVar-1], '-', color=color,     label=Label1)
    ax1.plot(tPred, yMat[:,iVar-1],     ':', color='#DF7401', label=Label2)

    color = ColorVec[0]
    ax2.plot(tOrig, yMatOrig[:,iVar], '-', color=color)
    ax2.plot(tPred, yMat[:,iVar],     ':', color='#848484')
    
    if (iSim == 0):   
        MSE  = (yMat[1::] - yMatOrig[1::])**2#/y_Data[1::]**2
    else:
        MSE += (yMat[1::] - yMatOrig[1::])**2#/y_Data[1::]**2
    
        
    iSim += 1


ax1.set_xlabel('Time [s]', size=34)
ax1.set_ylabel('Displacement [m]', color=ColorVec[1], size=34)
ax1.tick_params(axis='y', labelcolor=ColorVec[1])
ax1.set_ylim([-6.,6.])

color = 'k'
ax2.set_ylabel('Velocity [m/s]', color=ColorVec[0], size=34)  # we already handled the x-label with ax1
ax2.tick_params(axis='y', labelcolor=ColorVec[0])
ax2.set_ylim([-6.,6.])

ax1.legend()

fig.tight_layout()  # otherwise the right y-label is slightly clipped

if (FigDir):
    #plt.savefig(FigDir+'/MSD_TrainedDynamics_DeepONet.eps', format='eps', bbox_inches='tight')
    plt.savefig(FigDir+'/MSD_TrainedDynamics_PINN.eps', format='eps', bbox_inches='tight')
    #plt.savefig(FigDir+'/MSD_TrainedDynamics_SVD.eps', format='eps', bbox_inches='tight')
    #plt.savefig(FigDir+'/MSD_TrainedDynamics_PCA_All.eps', format='eps', bbox_inches='tight')
    pass


MSE_ = MSE.sum(axis=0) / (iSim) / (MSE.shape[0])
print('Mean Squared Error = ', MSE_)

RMSE = np.sqrt( MSE.sum(axis=0) / ((iSim) * MSE.shape[0]) )
print('Root Mean Squared Error = ', RMSE)

In [ ]:
### EXPERIMENTING

# FileName  = WORKSPACE_PATH + '/ROMNet/Data/MSD_100Cases/Orig/test/ext/Input.csv'# + InputData.InputFile
# Data      = pd.read_csv(FileName, header=0)
# Input     = Data[branch_vars].to_numpy()
# VarNames  = list(Data.columns)
# Nu        = Input.shape[1]
# u         = []
# for iu in range(Nu):
#     u.append(np.unique(Input[:,iu]))
# u         = np.stack(u, axis=0).T



# CostVec_Int = []

# iSim = 0
# for iTest in [0,1,2,3,4]:
    
#     print('u0 = ', u[iTest,:])
    
#     t0 = time.time()
#     ### Generating Data  
#     output     = solve_ivp( fOrig, tout[[0,-1]], u[iTest,:], method=SOLVER, t_eval=tout, rtol=1.e-15, atol=1.e-10 )
#     t1 = time.time()
#     CostVec_Int.append(t1-t0)
    

#     iSim += 1

In [ ]:
# CostVec_Loop = []

# iTestVec     = [0,1,2,3,4]

# tLoop        = np.linspace(0.,15.,3)

# iSim = 0
# for iTest in iTestVec:
    
#     print('u0 = ', u[iTest,:])
    
#     t0 = time.time()
#     yMatLoop      = np.zeros((len(tLoop),2))
#     InputPredLoop = np.concatenate([u[iTest,:][np.newaxis,...], (tLoop[0])[np.newaxis,np.newaxis]], axis=1)
#     yMatLoop[0,:] = model.net.predict(InputPredLoop)
#     for it, tLoop_ in enumerate(tLoop[1:]):
#         InputPredLoop    = np.concatenate([yMatLoop[it,:][np.newaxis,...], (tLoop[it+1]-tout[it])[np.newaxis,np.newaxis]], axis=1)
#         yMatLoop[it+1,:] = model.net.predict(InputPredLoop)

#     t1 = time.time()
#     CostVec_Loop.append(t1-t0)
    
#     plt.plot(tLoop, yMatLoop[:,iVar-1], 'b')
#     plt.xlabel('t')
#     plt.ylabel(VarOI)
#     plt.legend()
#     #plt.show()
    
#     iSim += 1

In [ ]:
# Cost_Loop     = 0.

# iTestVec      = [0,1,2,3,4]*10
# n_test         = len(iTestVec)
    


# InputPredLoop = np.concatenate([u[iTestVec,:], np.array([tLoop[0]]*n_test)[...,np.newaxis]], axis=1)
# yMatLoop      = model.net.predict(InputPredLoop)
# yMatLoopAll   = []
# for it, tLoop_ in enumerate(tLoop[1:]):
#     InputPredLoop = np.concatenate([yMatLoop, np.array([tLoop[it+1]-tLoop[it]]*n_test)[...,np.newaxis]], axis=1)
#     t0            = time.time()
#     yMatLoop      = model.net.predict(InputPredLoop)
#     t1            = time.time()
#     Cost_Loop    += t1-t0
#     yMatLoopAll.append(yMatLoop)

## Plotting Loss Histories

In [ ]:
Data = pd.read_csv(path_to_run_fld+'/Training/History.csv')

fig  = plt.figure(figsize=(12,8))
plt.plot(Data['tot_loss'],     label='Training')
plt.plot(Data['val_tot_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Tot. Loss [MSE]')
plt.legend()
plt.yscale('log')

fig  = plt.figure(figsize=(12,8))
plt.plot(Data['pts_loss'],     label='Training')
plt.plot(Data['val_pts_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Data Loss [MSE]')
plt.legend()
plt.yscale('log')
#plt.ylim([1.e-2, 1.e0])

# fig  = plt.figure(figsize=(12,8))
# plt.plot(Data['res_loss'],     label='Training')
# plt.plot(Data['val_res_loss'], label='Validation')
# plt.xlabel('Epoch')
# plt.ylabel('Residual Loss [MSE]')
# plt.legend()
# plt.yscale('log')

# fig  = plt.figure(figsize=(12,8))
# plt.plot(Data['ics_loss'],     label='Training')
# plt.plot(Data['val_ics_loss'], label='Validation')
# plt.xlabel('Epoch')
# plt.ylabel('I.C.s Loss [MSE]')
# plt.legend()
# plt.yscale('log')

## Evaluating Branches' and Trunk's Outputs

In [ ]:
FileName   = InputData.path_to_data_fld + '/test/ext/Input.csv'# + InputData.InputFile
DataIn     = pd.read_csv(FileName, header=0)
Input_     = DataIn.to_numpy()
VarNames   = list(DataIn.columns)

FileName   = InputData.path_to_data_fld + '/test/ext/Output.csv'# + InputData.InputFile
Data       = pd.read_csv(FileName, header=0)
Output_    = Data.to_numpy()

In [ ]:
FileName  = WORKSPACE_PATH + '/ROMNet/Data/MSD_100Cases/Orig/test/ext/Input.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
Input     = Data[branch_vars].to_numpy()
VarNames  = list(Data.columns)
Nu        = Input.shape[1]
u         = []
for iu in range(Nu):
    u.append(np.unique(Input[:,iu]))
u         = np.stack(u, axis=0).T
n_test     = u.shape[0]


## Variable to Be Visualized
VarOI     = 'v'
for iVar in range(1,len(VarNames)):
    if (VarNames[iVar] == VarOI):
        print('iVar = ', iVar)
        break

iSim = 0
for iTest in range(n_test):#[0,1,2,3,4]:
    
#     ### Generating Data  
#     output     = solve_ivp( fOrig, tout[[0,-1]], u[iTest,:], method=SOLVER, t_eval=tout, rtol=1.e-15, atol=1.e-10 )
#     tOrig      = tout
#     yMatOrig   = output.y.T
    
    ### Creating NN Input
    tPred     = tOrig[...,np.newaxis]
    Nt        = len(tPred)
    uPred     = u[iTest,:]*np.ones((Nt,2))
    InputPred = np.concatenate((uPred,tPred), axis=1)

In [ ]:
iVar     = 1

output_1 = model.net.system_of_components['DeepONet'].components['Branch_1'].call(u, model.net.layers_dict, None, training=False)
output_2 = model.net.system_of_components['DeepONet'].components['Branch_2'].call(u, model.net.layers_dict, None, training=False)

fig = plt.figure(figsize=(12,12))
ax  = fig.add_subplot(projection='3d')
plt.plot(u[:,0], u[:,1], output_1[:,0], 'b-')
plt.plot(u[:,0], u[:,1], output_1[:,1], 'b:')
plt.plot(u[:,0], u[:,1], output_2[:,0], 'g-')
plt.plot(u[:,0], u[:,1], output_2[:,1], 'g:')
#plt.xlim([1.e-6, 1.e-2])
plt.xlabel(r'x')
plt.ylabel(r'v')
plt.ylabel(r'$\alpha_i$')

In [ ]:
fig, ax1 = plt.subplots()
ax2      = ax1.twinx() 

output_  = model.net.system_of_components['DeepONet'].components['Trunk_1'].call(tPred, model.net.layers_dict, None, training=False)
ax1.plot(tPred, output_[:,0], '-', color=ColorVec[1], label=Label1)
ax1.plot(tPred, output_[:,1], '-', color=ColorVec[1], label=Label2)

output_  = model.net.system_of_components['DeepONet'].components['Trunk_2'].call(tPred, model.net.layers_dict, None, training=False)
color    = ColorVec[0]
ax2.plot(tPred, output_[:,0], '-', color=ColorVec[0])
ax2.plot(tPred, output_[:,1], '-', color=ColorVec[0])

ax1.set_xlabel('Time [s]', size=34)
ax1.set_ylabel(r'$\psi_{x_i}$', color=ColorVec[1], size=34)
ax1.tick_params(axis='y', labelcolor=ColorVec[1])
ax1.set_ylim([-3.5,3.5])

color = 'k'
ax2.set_ylabel(r'$\psi_{v_i}$', color=ColorVec[0], size=34)  # we already handled the x-label with ax1
ax2.tick_params(axis='y', labelcolor=ColorVec[0])
#ax2.set_ylim([-4.5,4.5])


fig.tight_layout()  # otherwise the right y-label is slightly clipped

if (FigDir):
    plt.savefig(FigDir+'//MSD_Trunk_Modes.eps', format='eps', bbox_inches='tight')